In [1]:
%load_ext autoreload
%autoreload 2

import json
import os.path as osp
import os,sys
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable

from torch_geometric.utils import to_undirected
from torch_cluster import radius_graph, knn_graph
from torch_geometric.datasets import MNISTSuperpixels
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from tqdm import tqdm
import argparse
import warnings
warnings.simplefilter('ignore')
from time import strftime, gmtime

sys.path.append('../')
import utils
import model.net as net
import model.data_loader as data_loader
from evaluate import evaluate

In [2]:
torch.__version__

'1.10.2+cu102'

In [3]:
!python --version

Python 3.8.8


### Restore ckpts

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
prefix = '/export/home/phys/kyungmip/L1DeepMETv2/ckpts_April30_scale_sigmoid/'
restore_ckpt = osp.join(prefix, 'last.pth.tar')

In [6]:
scale_momentum = 1
norm = torch.tensor([1./scale_momentum, 1./scale_momentum, 1./scale_momentum, 1., 1., 1.]).to(device)

In [7]:
model_new = net.Net(continuous_dim=6, categorical_dim=2 , norm=norm).to(device)

In [8]:
param_restored_new = utils.load_checkpoint(restore_ckpt, model_new)
param_restored_new

{'epoch': 62,
 'state_dict': OrderedDict([('graphnet.embed_charge.weight',
               tensor([[ 0.9152, -1.0589, -0.9910,  0.0120, -1.2543,  0.2998, -0.3684, -0.0260],
                       [ 1.1699, -1.1929, -0.4268, -0.7047, -0.3582,  0.5368,  1.0060, -0.7781],
                       [-1.1897, -0.7680,  0.9429,  0.2915, -0.2274, -1.3632,  0.6982,  0.4960]],
                      device='cuda:0')),
              ('graphnet.embed_pdgid.weight',
               tensor([[-2.0383, -0.3847, -0.2413,  0.9122, -0.4805,  0.2302, -0.4746,  0.9317],
                       [-0.4117,  1.3556,  0.8550,  0.9977, -0.1622,  1.0582, -0.3220, -0.1858],
                       [ 0.3344, -1.7237,  0.2500,  0.0249, -1.0838, -0.7614, -0.2512, -1.4239],
                       [-0.0926,  0.4536, -0.2328,  1.5914, -0.3767, -1.6118,  0.4281,  0.3517],
                       [-0.8126, -1.1399, -0.2381, -0.4232,  0.0113, -1.9791, -0.4646, -1.3267],
                       [ 0.1623,  0.1171, -0.4400,  0.8133, -

### Import data

In [9]:
'''
pre_fix = '/export/home/phys/kyungmip/L1DeepMETv2/'

data_dir = pre_fix + 'data_ttbar/'        # name of the input data folder

dataloaders = data_loader.fetch_dataloader(data_dir = data_dir, batch_size=1, validation_split=.2)

train_dl = dataloaders['train']
test_dl = dataloaders['test']

print('Training dataloader: {}, Test dataloader: {}'.format(len(train_dl), len(test_dl)))

torch.save(test_dl, 'test_dataloader_per_event.pth')
'''

"\npre_fix = '/export/home/phys/kyungmip/L1DeepMETv2/'\n\ndata_dir = pre_fix + 'data_ttbar/'        # name of the input data folder\n\ndataloaders = data_loader.fetch_dataloader(data_dir = data_dir, batch_size=1, validation_split=.2)\n\ntrain_dl = dataloaders['train']\ntest_dl = dataloaders['test']\n\nprint('Training dataloader: {}, Test dataloader: {}'.format(len(train_dl), len(test_dl)))\n\ntorch.save(test_dl, 'test_dataloader_per_event.pth')\n"

In [10]:
test_dl = torch.load('test_dataloader_per_event.pth')
#test_dl = torch.load('test_dataloader.pth')

In [11]:
# tensor operations
def getdot(vx, vy):
    return torch.einsum('bi,bi->b',vx,vy)

def getscale(vx):
    return torch.sqrt(getdot(vx,vx))

def scalermul(a,v):
    return torch.einsum('b,bi->bi',a,v)

In [12]:
def metric(weights, particles_vis, genMET, batch, scale_momentum = 1.):
    # qT is the genMET
    qTx = genMET[:,0]
    qTy = genMET[:,1]

    v_qT=torch.stack((qTx,qTy),dim=1)

    # momentum of visible particles
    px = particles_vis[:,1]
    py = particles_vis[:,2]

    # regressed uT: momentum of the system of all visible particles
    uTx = scatter_add(weights*px, batch) 
    uTy = scatter_add(weights*py, batch) 
    
    # regressed MET
    METx = (-1) * uTx * scale_momentum
    METy = (-1) * uTy * scale_momentum

    v_MET=torch.stack((METx, METy),dim=1)

    # PUPPI MET using PUPPI weights
    wgt_puppi = particles_vis[:,5]

    puppiMETx = (-1)*scatter_add(wgt_puppi*px, batch) * scale_momentum
    puppiMETy = (-1)*scatter_add(wgt_puppi*py, batch) * scale_momentum
   
    v_puppiMET = torch.stack((puppiMETx, puppiMETy),dim=1)

    def compute(vector):
        response = getdot(vector,v_qT)/getdot(v_qT,v_qT)
        v_paral_predict = scalermul(response, v_qT)
        u_paral_predict = getscale(v_paral_predict)-getscale(v_qT)
        v_perp_predict = vector - v_paral_predict
        u_perp_predict = getscale(v_perp_predict)
        return [u_perp_predict.cpu().detach().numpy(), u_paral_predict.cpu().detach().numpy(), response.cpu().detach().numpy()]

    resolutions = {
        'MET':      compute(v_MET),
        'puppiMET': compute(v_puppiMET)
    }

    
    # gen MET, regressed MET, and PUPPI MET
    METs = {
        'genMETx': qTx.cpu().detach().numpy(),
        'genMETy': qTy.cpu().detach().numpy(),
        'genMET': getscale(v_qT).cpu().detach().numpy(),
        
        'METx': METx.cpu().detach().numpy(),
        'METy': METy.cpu().detach().numpy(),
        'MET': getscale(v_MET).cpu().detach().numpy(),
        
        'puppiMETx': puppiMETx.cpu().detach().numpy(),
        'puppiMETy': puppiMETy.cpu().detach().numpy(),
        'puppiMET': getscale(v_puppiMET).cpu().detach().numpy()
    }
    
    return resolutions, METs

In [13]:
qT_arr = []
    
MET_arr = {
    'genMETx': [],
    'genMETy': [],
    'genMET': [],
        
    'METx': [],
    'METy': [],
    'MET': [],
        
    'puppiMETx': [],
    'puppiMETy': [],
    'puppiMET': []
}
    
resolutions_arr = {
    'MET':      [[],[],[]],
    'puppiMET': [[],[],[]],
}


In [ ]:
from torch_scatter import scatter_add

with tqdm(total=len(test_dl)) as t:
    for data in test_dl:
        data = data.to(device)
            
        x_cont = data.x[:,:6]       # include puppi
        x_cat = data.x[:,6:].long()

        etaphi = torch.cat([data.x[:,3][:,None], data.x[:,4][:,None]], dim=1)

        edge_index = radius_graph(etaphi, r=0.4, batch=data.batch, loop=False, max_num_neighbors=255)  # turn off self-loop
            
        result = model_new(x_cont, x_cat, edge_index, data.batch) # ML weights to regress MET
        
        # compute all metrics on this batch
        resolutions, METs = metric(result, data.x, data.y, data.batch)
        
        for key in resolutions_arr.keys():
            for i in range(len(resolutions_arr[key])):
                resolutions_arr[key][i]=np.concatenate((resolutions_arr[key][i],resolutions[key][i]))

        for key in MET_arr.keys():
            MET_arr[key]=np.concatenate((MET_arr[key],METs[key]))

  0%|          | 0/199708 [00:00<?, ?it/s]